<a href="https://colab.research.google.com/github/ampnb/Mind/blob/main/MW_Bass01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.2 MB/s eta 0:00:00


In [34]:
!pip install pyeeg

In [35]:
import mne
import numpy as np
from matplotlib import pyplot as plt

In [36]:
#Load the .ftf files
raw_meditation = mne.io.read_raw_fif('bass02_meditation_trial1.fif', preload=True)
raw_mindwandering = mne.io.read_raw_fif('bass02_mindwandering_trial1.fif', preload=True)

Opening raw data file bass02_meditation_trial1.fif...
    Range : 0 ... 304266 =      0.000 ...  1217.064 secs
Ready.
Reading 0 ... 304266  =      0.000 ...  1217.064 secs...
Opening raw data file bass02_mindwandering_trial1.fif...
    Range : 0 ... 313755 =      0.000 ...  1255.020 secs
Ready.
Reading 0 ... 313755  =      0.000 ...  1255.020 secs...


<ipython-input-36-2d0b0df2413b>:2: RuntimeWarning: This filename (bass02_meditation_trial1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_meditation = mne.io.read_raw_fif('bass02_meditation_trial1.fif', preload=True)
<ipython-input-36-2d0b0df2413b>:3: RuntimeWarning: This filename (bass02_mindwandering_trial1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_mindwandering = mne.io.read_raw_fif('bass02_mindwandering_trial1.fif', preload=True)


In [37]:
# #Plot
# raw_meditation.plot()
# raw_mindwandering.plot()

In [38]:
#Filter the data
raw_meditation.filter(l_freq=1., h_freq=50.)
raw_mindwandering.filter(l_freq=1., h_freq=50.)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (3.300 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 5

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished


<Raw | bass02_mindwandering_trial1.fif, 8 x 313756 (1255.0 s), ~19.2 MB, data loaded>

In [39]:
# Run ICA for artifact rejection
ica = mne.preprocessing.ICA(n_components=8, random_state=0)
ica.fit(mne.concatenate_raws([raw_meditation, raw_mindwandering]))

raw_meditation = ica.apply(raw_meditation)
raw_mindwandering = ica.apply(raw_mindwandering)

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components
Fitting ICA took 6.4s.
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components
Applying ICA to Raw instance
    Transforming to ICA space (8 components)
    Zeroing out 0 ICA components
    Projecting back using 8 PCA components


In [40]:
# # Plot the raw data after ICA
# raw_meditation.plot()
# raw_mindwandering.plot()

In [41]:
# Define epoch parameters
tmin, tmax = 0., 1.  # start and end of each epoch in seconds

# Define the number of seconds per epoch
epoch_secs = 1.

# The number of samples per epoch:
epoch_samps = int(epoch_secs * raw_meditation.info['sfreq'])

# Number of total epochs we'll extract from each dataset:
n_epochs_meditation = int(np.floor(len(raw_meditation.times) / epoch_samps))
n_epochs_mindwandering = int(np.floor(len(raw_mindwandering.times) / epoch_samps))

# Now we'll create the epochs:
data_meditation = raw_meditation.get_data()
data_mindwandering = raw_mindwandering.get_data()

epochs_meditation = []
for i in range(n_epochs_meditation):
    start = i * epoch_samps
    stop = (i + 1) * epoch_samps
    epochs_meditation.append(data_meditation[:, start:stop])

epochs_mindwandering = []
for i in range(n_epochs_mindwandering):
    start = i * epoch_samps
    stop = (i + 1) * epoch_samps
    epochs_mindwandering.append(data_mindwandering[:, start:stop])

# Convert lists of epochs to numpy arrays for easier handling later
epochs_meditation = np.array(epochs_meditation)
epochs_mindwandering = np.array(epochs_mindwandering)


In [42]:
def compute_alpha_power(epochs, sfreq):
    # Define alpha band
    low, high = 8, 12

    # Compute FFT
    freqs = np.fft.rfftfreq(epochs.shape[-1], d=1./sfreq)
    fft_vals = np.abs(np.fft.rfft(epochs, axis=-1))

    # Find indices corresponding to alpha band
    idx_alpha = np.where((freqs >= low) & (freqs <= high))[0]

    # Compute mean power in alpha band
    alpha_power = fft_vals[..., idx_alpha].mean(axis=-1)

    return alpha_power

# Compute alpha power for each epoch
alpha_power_meditation = compute_alpha_power(epochs_meditation, raw_meditation.info['sfreq'])
alpha_power_mindwandering = compute_alpha_power(epochs_mindwandering, raw_mindwandering.info['sfreq'])


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create data and labels
X = np.concatenate([alpha_power_meditation, alpha_power_mindwandering])
y = np.array([0]*len(alpha_power_meditation) + [1]*len(alpha_power_mindwandering))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Classification report for Random Forest:")
print(classification_report(y_test, y_pred_rf))

# Non-Linear SVM
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("\nClassification report for Non-Linear SVM:")
print(classification_report(y_test, y_pred_svm))

# Logistic Regression
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("\nClassification report for Logistic Regression:")
print(classification_report(y_test, y_pred_lr))


Classification report for Random Forest:
              precision    recall  f1-score   support

           0       0.67      0.92      0.78       506
           1       0.25      0.06      0.09       240

    accuracy                           0.64       746
   macro avg       0.46      0.49      0.44       746
weighted avg       0.54      0.64      0.56       746


Classification report for Non-Linear SVM:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81       506
           1       0.00      0.00      0.00       240

    accuracy                           0.68       746
   macro avg       0.34      0.50      0.40       746
weighted avg       0.46      0.68      0.55       746


Classification report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81       506
           1       0.00      0.00      0.00       240

    accuracy                           0.68       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Higuchi

In [48]:
pip install hfda

  Preparing metadata (setup.py) ... done
  Created wheel for hfda: filename=hfda-0.1.1-py3-none-any.whl size=2139 sha256=a81d0f8e5d4c624d8579e4bfbff789fcd4ea7804096160f81a94d96718ab1573
  Stored in directory: /root/.cache/pip/wheels/ae/2c/72/48f1edf94fa2bf74d8896561391b539909292865a1391aebdd
Successfully built hfda


In [49]:
import numpy as np
import mne
import hfda

def compute_hfd_for_each_channel(raw_data, k_max):
    # Get data from Raw object
    data, times = raw_data[:]

    # Compute HFD for each channel
    hfd_values = np.array([hfda.measure(channel_data, k_max) for channel_data in data])

    return hfd_values

# Maximum k for HFD
k_max = 5

# Load the .fif files
raw_meditation = mne.io.read_raw_fif('bass02_meditation_trial1.fif', preload=True)
raw_mindwandering = mne.io.read_raw_fif('bass02_mindwandering_trial1.fif', preload=True)

# Compute HFD for each channel for each condition
hfd_meditation = compute_hfd_for_each_channel(raw_meditation, k_max)
hfd_mindwandering = compute_hfd_for_each_channel(raw_mindwandering, k_max)

Opening raw data file bass02_meditation_trial1.fif...
    Range : 0 ... 304266 =      0.000 ...  1217.064 secs
Ready.
Reading 0 ... 304266  =      0.000 ...  1217.064 secs...
Opening raw data file bass02_mindwandering_trial1.fif...
    Range : 0 ... 313755 =      0.000 ...  1255.020 secs
Ready.
Reading 0 ... 313755  =      0.000 ...  1255.020 secs...


<ipython-input-49-85db815d58c4>:18: RuntimeWarning: This filename (bass02_meditation_trial1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_meditation = mne.io.read_raw_fif('bass02_meditation_trial1.fif', preload=True)
<ipython-input-49-85db815d58c4>:19: RuntimeWarning: This filename (bass02_mindwandering_trial1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_mindwandering = mne.io.read_raw_fif('bass02_mindwandering_trial1.fif', preload=True)


In [50]:
pip install nolds pyefd


# Detrended Fluctuation Analysis (DFA)

In [55]:
import nolds

# Get the data from the MNE Raw object
data, times = raw_meditation.get_data(return_times=True)

# Compute DFA for each channel
for i, channel_data in enumerate(data):
    alpha = nolds.dfa(channel_data)
    print(f'DFA alpha for channel {i+1}: {alpha}')



DFA alpha for channel 1: 1.6747522170213065
DFA alpha for channel 2: 1.5348684324761481
DFA alpha for channel 3: 1.5754153313713244
DFA alpha for channel 4: 1.5518711121192026
DFA alpha for channel 5: 1.569262767170263
DFA alpha for channel 6: 1.535570653667137
DFA alpha for channel 7: 1.7691293917542465
DFA alpha for channel 8: 1.6121688339807632


In [66]:
!pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-7da8nohi
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-7da8nohi
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28114 sha256=dd3e73a5ba03082341717101d245895613e68161862e05fa197c3ca489f13cd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-bct797yu/wheels/a8/c4/1a/cee09dcc12a11620066d35ace42e3c1e3bfbcc1db3a0ce7788
Successfully built pyeeg
  Attempting uninstall: pyeeg
    Found existing installation: pyeeg 0.0.2
    Uninstalling pyeeg-0.0.2:
      Successfully uninstalled pyeeg-0.0.2


# Petitjean Fractal Dimension (PFD)

In [67]:
import pyeeg

# Get the data from the MNE Raw object
data, times = raw_meditation.get_data(return_times=True)

# Compute PFD for each channel
for i, channel_data in enumerate(data):
    pfd = pyeeg.pfd(channel_data)
    print(f'PFD for channel {i+1}: {pfd}')



PFD for channel 1: 0.5387028416255876
PFD for channel 2: 0.5383421575727636
PFD for channel 3: 0.5392007231679988
PFD for channel 4: 0.5373809743799763
PFD for channel 5: 0.5384908702213497
PFD for channel 6: 0.5375407654143107
PFD for channel 7: 0.5372223855986799
PFD for channel 8: 0.5373359137945707


# Sample Entropy

In [ ]:
import nolds
from pyeeg import ap_entropy

# Constants for Approximate Entropy
m = 2  # The length of compared run of data
r = 0.2 * np.std(data)  # The tolerance (generally 0.1...0.25 times the standard deviation of the data)

# Get the data from the MNE Raw object
meditation_data, _ = raw_meditation.get_data(return_times=True)
mindwandering_data, _ = raw_mindwandering.get_data(return_times=True)

for i in range(meditation_data.shape[0]):  # Assuming your data is 2D with the shape (channels, time)
    channel_data_meditation = meditation_data[i]
    channel_data_mindwandering = mindwandering_data[i]

    # Calculate Sample Entropy
    sampen_meditation = nolds.sampen(channel_data_meditation)
    sampen_mindwandering = nolds.sampen(channel_data_mindwandering)

    print(f'Sample Entropy for Meditation channel {i+1}: {sampen_meditation}')
    print(f'Sample Entropy for Mind-Wandering channel {i+1}: {sampen_mindwandering}')

    # Calculate Approximate Entropy
    apen_meditation = ap_entropy(channel_data_meditation, m, r)
    apen_mindwandering = ap_entropy(channel_data_mindwandering, m, r)

    print(f'Approximate Entropy for Meditation channel {i+1}: {apen_meditation}')
    print(f'Approximate Entropy for Mind-Wandering channel {i+1}: {apen_mindwandering}')
